In [1]:
import os
import xml.etree.ElementTree as ET

In [2]:
class_mapping = {
    "tire": 0,
    "spring fish trap": 1,
    "circular fish trap": 2,
    "rectangular fish trap": 3,
    "eel fish trap": 4,
    "fish net": 5,
    "wood": 6,
    "rope": 7,
    "bundle of ropes": 8
}

In [ ]:
# XML 파일을 YOLO 형식의 txt로 변환하는 함수
def convert_xml_to_yolo(xml_folder, output_folder):
    # 출력 폴더가 없으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # XML 폴더 내의 모든 파일을 확인
    for xml_file in os.listdir(xml_folder):
        # XML 파일이 아닌 경우 건너뛰기
        if not xml_file.endswith(".xml"):
            continue
        
        # XML 파일 파싱
        tree = ET.parse(os.path.join(xml_folder, xml_file))
        root = tree.getroot()

        # 이미지 크기 정보 가져오기
        size = root.find('size')
        img_width = int(size.find('width').text)  # 이미지 너비
        img_height = int(size.find('height').text)  # 이미지 높이

        # YOLO 형식의 라벨 파일 이름 생성
        label_file_name = os.path.splitext(xml_file)[0] + ".txt"
        label_file_path = os.path.join(output_folder, label_file_name)

        # 라벨 파일 열기
        with open(label_file_path, "w") as label_file:
            # 모든 객체를 탐색
            for obj in root.iter('object'):
                class_name = obj.find('name').text 
                if class_name not in class_mapping:
                    continue 

                # 클래스 ID 가져오기
                class_id = class_mapping[class_name]

                # 바운딩 박스 정보 가져오기
                bbox = obj.find('bndbox')
                xmin = float(bbox.find('xmin').text) 
                ymin = float(bbox.find('ymin').text) 
                xmax = float(bbox.find('xmax').text) 
                ymax = float(bbox.find('ymax').text) 

                # YOLO 형식으로 변환
                x_center = ((xmin + xmax) / 2) / img_width 
                y_center = ((ymin + ymax) / 2) / img_height 
                width = (xmax - xmin) / img_width 
                height = (ymax - ymin) / img_height 
                
                # 좌표 값이 [0, 1] 범위에 있는지 확인하고, 범위를 벗어나면 건너뜀
                if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
                    print(f"Warning: Non-normalized or out of bounds coordinates in {xml_file}, skipping object.")
                    continue
                
                # 라벨 파일에 클래스 ID와 좌표 정보를 저장
                label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

In [4]:
root_folder = "./marine-litter-data"
train_xml_folder = root_folder + "/train/labels_xml/"
val_xml_folder = root_folder + "/val/labels_xml/"
train_output_folder = root_folder + "/train/labels/"
val_output_folder = root_folder + "/val/labels/"    

In [5]:
convert_xml_to_yolo(train_xml_folder, train_output_folder)
convert_xml_to_yolo(val_xml_folder, val_output_folder)